In [1]:
import json
import sqlite3
import csv

M0: Para esta parte, me guié del código entregado en la pauta del taller 3b.

In [2]:
with open('movies.json',encoding = 'utf8') as movies_file:
    movies = json.load(movies_file)

In [3]:
connection = sqlite3.connect('movies.db')
cursor = connection.cursor()

cursor.execute("CREATE TABLE Movies(mid INTEGER PRIMARY KEY, mtitle TEXT, myear INTEGER)")
cursor.execute("CREATE TABLE Actors(aid INTEGER PRIMARY KEY, aname TEXT)")
cursor.execute("CREATE TABLE Genres(gid INTEGER PRIMARY KEY, genre TEXT)")
cursor.execute("CREATE TABLE ActorsMovies(actor_id INTEGER, movie_id INTEGER, FOREIGN KEY (actor_id) REFERENCES Actors, FOREIGN KEY (movie_id) REFERENCES Movies)")
cursor.execute("CREATE TABLE GenresMovies(genre_id INTEGER, movie_id INTEGER, FOREIGN KEY (genre_id) REFERENCES Genres, FOREIGN KEY (movie_id) REFERENCES Movies)")

connection.commit()
connection.close()

In [4]:
connection = sqlite3.connect('movies.db')
cursor = connection.cursor()

mid = 1
aid = 1
gid = 1
genres = {}
actors = {}

for movie in movies:

    mtitle = movie["title"]
    myear = movie["year"]

    cursor.execute("INSERT INTO Movies VALUES (?,?,?)", (mid, mtitle, myear))

    for genre in movie["genres"]:
        if genre not in genres:
            genres[genre] = gid
            cursor.execute("INSERT INTO Genres VALUES (?,?)", (gid, genre))
            gid += 1
        cursor.execute("INSERT INTO GenresMovies VALUES (?,?)", (genres[genre], mid))

    for actor in movie["cast"]:
        if actor != "." and actor != "]" and actor != "Academy Award for Best Supporting Actress" and actor != "(" and actor != ")" and actor != "$119" and actor != "and" and actor != "500" and actor != "000" and actor != "[1]" and actor != "[2]" and actor != "$116" and actor!= "[3]" and actor != "[4]" and actor != "[5]" and actor != "[6]" and actor != "[7]" and actor != "[8]" and actor != "[9]" and actor != "[10]" and actor != "$89" and actor != "$86" and actor != "273" and actor != "333" and actor != "$79" and actor != "666" and actor != "653" and actor != "$47" and actor != "542" and actor != "841" and actor != "285" and actor != "152" and actor != "$45" and actor != "$43" and actor != "411" and actor != "063" and actor != "008" and actor != "075" and actor != "$39" and actor != "552" and actor != "the life of Jonathan Caouette" and actor != "the life and death of" and actor != "11" and actor != "and its relations with the" and actor != "in" and actor != "a" and actor != "by" and actor != "-":
            if actor not in actors:
                actors[actor] = aid
                cursor.execute("INSERT INTO Actors VALUES (?,?)", (aid, actor))
                aid += 1
            cursor.execute("INSERT INTO ActorsMovies VALUES (?,?)", (actors[actor], mid))

    mid += 1

connection.commit()
connection.close()

M1

In [5]:
import sys #tuve que usar esto para poder importar pyrematch, ya que con !pip3 no me funcionó
!{sys.executable} -m pip install pyrematch
!{sys.executable} -m pip install pyrematch

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.0.11/libexec/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.0.11/libexec/bin/python3 -m pip install --upgrade pip' command.


In [6]:
import urllib.request as req
import ssl
import pyrematch as re
import bs4
import sqlite3

In [7]:
connection = sqlite3.connect('movies.db')
cursor = connection.cursor()
cursor.execute('select * from Actors')
lista_actores = []
for reg in cursor:
    lista_actores.append(reg[1])
cursor.execute('select * from Movies')
lista_peliculas = []
for reg in cursor:
    lista_peliculas.append(reg[1])
connection.commit()
connection.close()

In [8]:
lista_actores

['Paul Boyton',
 'Ching Ling Foo',
 'May Clark',
 'William Carrington',
 'J. Stuart Blackton',
 'Florence Lawrence',
 'William S. Hart',
 'William Craven',
 'Unknown',
 'Bertha Regustus',
 'Edward Boulden',
 'Arthur V. Johnson',
 'Linda Arvidson',
 'William V. Ranous',
 'George Gebhardt',
 'Charles Inslee',
 'D. W. Griffith',
 'Harry Solter',
 'Tom Ricketts',
 'Edward Dillon',
 'Hobart Bosworth',
 'Betty Harte',
 'Florence Auer',
 'John G. Adolfi',
 'Paul Panzer',
 'John Tansey',
 'Marion Leonard',
 'Owen Moore',
 'Mack Sennett',
 "Barry O'Moore",
 'John R. Cumpson',
 'Anita Hendrie',
 'Leopold Wharton',
 'Mary Pickford',
 'George Nichols',
 'Gladys Egan',
 'Marc McDermott',
 'Charles Stanton Ogle',
 'Robert Z. Leonard',
 'Charles Craig',
 'Augustus Phillips',
 'Mary Fuller',
 'Kate Bruce',
 'Harry Carey',
 'Henry B. Walthall',
 'Frank Powell',
 'Charles West',
 'Sidney Olcott',
 'Gene Gauntier',
 "Thomas O'Connor",
 'Anna Rosemond',
 'George Barnes',
 'Frank H. Crane',
 'Tom Mix',
 'S

In [9]:
lista_peliculas

['After Dark in Central Park',
 "Boarding School Girls' Pajama Parade",
 "Buffalo Bill's Wild West Parad",
 'Caught',
 'Clowns Spinning Hats',
 'Capture of Boer Battery by British',
 'The Enchanted Drawing',
 'Feeding Sea Lions',
 'How to Make a Fat Wife Out of Two Lean Ones',
 'New Life Rescue',
 'New Morning Bath',
 'Searching Ruins on Broadway, Galveston, for Dead Bodies',
 'The Tribulations of an Amateur Photographer',
 "Trouble in Hogan's Alley",
 'Two Old Sparks',
 'The Wonder, Ching Ling Foo',
 'Watermelon Contest',
 'Acrobats in Cairo',
 'An Affair of Honor',
 'Another Job for the Undertaker',
 'Arrival of Tongkin Train',
 "The Artist's Dilemma",
 'Band and Battalion of the U.S. Indian School',
 "Barnum and Bailey's Circus",
 'Beef Extract Room',
 'Boxing in Barrels',
 'Branding Hams',
 'Buffalo Street Parade',
 "A Busy Corner at Armour's",
 'The Bund, Shanghai',
 'Circular Panorama of the Base of the Electric Tower, Ending Looking Down the Mall',
 'Circular Panorama of the Ele

In [10]:
def buscar_fecha(trs):
    patron = '(>)!r{Born}(<)'
    patron_f = "(^|\D)!f{\d+{1,4}\-\d+{1,2}\-\d+{1,4}}($|\D)" 
    regex   = re.compile(patron)
    regex_f = re.compile(patron_f)
    for tr in trs:
        m = regex.find(str(tr))
        if m:
            td = tr.find('td')
            m_f = regex_f.find(td.text)
            if m_f:
                return m_f.group('f')
            else:
                return td.text.strip()
    return ''

In [11]:
def buscar_director(trs):
    patron = '(>)!r{Directed by}(<)'
    regex   = re.compile(patron)
    for tr in trs:
        m = regex.find(str(tr))
        if m:
            td = tr.find('td')
            return td.text.strip()
    return ''

In [12]:
#se hizo prueba con 50 ultimos ya que hay mas coincidencias
headers = {'User-Agent':'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}

In [13]:
def obtener_texto(link):
    respuesta = req.Request(link,headers=headers)
    return req.urlopen(respuesta,context=ssl.SSLContext()).read().decode('utf-8')
#obtener_texto('https://en.wikipedia.org/wiki/{}'.format(lista_peliculas[-50].replace(' ','_')))

Usé el query de Wikipedia y no su librería directamente, ya que descubrí que así se demoraba la mitad. Lo tuve que hacer, porque mi computador es demasiado lento y me estresaba que se congelara cada vez que probaba una consulta

In [14]:
directores_peli = []
for pelicula in lista_peliculas[-50:]:
    try:
        link = 'https://en.wikipedia.org/wiki/{}'.format(pelicula.replace(' ','_'))
        texto = obtener_texto(link)
        #en las ramas tr hay relaciones de tabla 
        #ej: nace -> 21-05-1815
        #   tr{th -> td}
        #tr alverga a th y td
        sopa = bs4.BeautifulSoup(texto)
        #todos los tr #tabla linea por linea
        trs = sopa.findAll('tr')
        directores_peli.append(buscar_director(trs).replace("'",""))
    except:
        directores_peli.append('')

In [15]:
directores_peli

['Ari Sandel',
 '',
 '',
 '',
 'Marielle Heller',
 '',
 'Jonah Hill',
 'Elizabeth Chomko',
 '',
 '',
 'Dario Argento',
 '',
 'Lasse Hallström\nJoe Johnston',
 '',
 '',
 'Joel Edgerton',
 'Matthew Heineman',
 'Joseph Kahn',
 '',
 '',
 '',
 '',
 '',
 'David Yates',
 '',
 'Sean Anders',
 'Rich Moore\nPhil Johnston',
 'Steven Caple Jr.',
 '',
 '',
 'Yorgos Lanthimos',
 'John McPhail',
 '',
 '',
 'David Robert Mitchell',
 '',
 '',
 'Peter Hedges',
 '',
 'Bob Persichetti\nPeter Ramsey\nRodney Rothman',
 '',
 '',
 '',
 'Rob Marshall',
 '',
 '',
 'Robert Zemeckis',
 'Etan Cohen',
 'Mimi Leder',
 '']

In [16]:
nacimiento = []
for actor in lista_actores[-50:]:
    actor = actor.replace(' ','_')
    link = f'https://en.wikipedia.org/wiki/{actor}'
    try:
        texto = obtener_texto(link)
        sopa = bs4.BeautifulSoup(texto)
        trs = sopa.findAll('tr')
        nacimiento.append(buscar_fecha(trs).replace("'",""))
    except:
        nacimiento.append('')

In [17]:
nacimiento

['',
 '1984-12-23',
 'Montpellier, France',
 '',
 '1971-03-24',
 '',
 '1984-02-23',
 '1991-07-07',
 '1975-10-07',
 '',
 '1974-01-30',
 '1998-04-29',
 '',
 '11 MarchEdinburgh, Scotland, UK[citation needed]',
 '',
 '',
 '',
 '1991-12-10',
 '1987-09-22',
 '1984-05-08',
 '1988-02-02',
 '1975-11-21',
 '1987-06-08',
 '1969-12-29',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '1988-12-27',
 '',
 'Dublin, Ireland',
 'Leila George DOnofrio1991/1992 (age\xa028–29)Sydney, New South Wales, Australia',
 '1945-03-24',
 '1982-08-26',
 '1989-06-27',
 '1964-01-07',
 '1982-06-29',
 '1970-06-15',
 '1985-06-25',
 '2005/2006 (age\xa015–16)',
 '1998-01-23',
 'Leipzig, East Germany',
 '',
 '1990-01-30']

In [18]:
connection = sqlite3.connect('movies.db')
cursor = connection.cursor()
cursor.execute('ALTER TABLE Actors ADD COLUMN fecha_naciemiento TEXT;')
cursor.execute('SELECT aid FROM Actors')
registros = list(cursor).copy()
for i in range(50):
    aid = registros[-50:][i][0]
    cursor.execute(f"update Actors set fecha_naciemiento='{nacimiento[i]}' WHERE aid={aid};")
cursor.execute('ALTER TABLE Movies ADD COLUMN director TEXT;')
cursor.execute('SELECT mid FROM Movies')
registros = list(cursor).copy()
for i in range(50):
    mid = registros[-50:][i][0]
    cursor.execute(f"update Movies set director='{directores_peli[i]}' WHERE mid={mid};")
connection.commit()
connection.close()

M2

In [19]:
def consultar(consulta):
    connection = sqlite3.connect('movies.db')
    cursor = connection.cursor()
    cursor.execute(consulta)
    for reg in list(cursor):
        print(reg)
    connection.commit()
    connection.close()

Consulta 1

In [20]:
consulta='select myear, count(*) as cuenta from Movies group by myear order by cuenta DESC LIMIT 3'
consultar(consulta)

(1919, 634)
(1925, 572)
(1936, 504)


Consulta 2

In [21]:
consulta ='select Actors.aname,(max(Movies.myear)-min(Movies.myear)) as trayectoria ' 
consulta+='from Actors, Movies, ActorsMovies where Actors.aid=ActorsMovies.actor_id '
consulta+='and ActorsMovies.movie_id=Movies.mid group by ActorsMovies.actor_id order by trayectoria DESC '
consulta+='LIMIT 5'
consultar(consulta)

('Harrison Ford', 98)
('Gloria Stuart', 80)
('Kenny Baker', 75)
('Lillian Gish', 75)
('Angela Lansbury', 74)


Consulta 3: esta consulta no la alcancé a hacer y preferí no ponerla incompleta. La cantidad de interacciones entre múltiples tablas para responderla nunca me cargaba en el computador. Lo siento :(

In [22]:
consulta ="select aname from Actors" #lo usé para chequear que no hubieran errores de formato en los nombres
consultar(consulta)

('Paul Boyton',)
('Ching Ling Foo',)
('May Clark',)
('William Carrington',)
('J. Stuart Blackton',)
('Florence Lawrence',)
('William S. Hart',)
('William Craven',)
('Unknown',)
('Bertha Regustus',)
('Edward Boulden',)
('Arthur V. Johnson',)
('Linda Arvidson',)
('William V. Ranous',)
('George Gebhardt',)
('Charles Inslee',)
('D. W. Griffith',)
('Harry Solter',)
('Tom Ricketts',)
('Edward Dillon',)
('Hobart Bosworth',)
('Betty Harte',)
('Florence Auer',)
('John G. Adolfi',)
('Paul Panzer',)
('John Tansey',)
('Marion Leonard',)
('Owen Moore',)
('Mack Sennett',)
("Barry O'Moore",)
('John R. Cumpson',)
('Anita Hendrie',)
('Leopold Wharton',)
('Mary Pickford',)
('George Nichols',)
('Gladys Egan',)
('Marc McDermott',)
('Charles Stanton Ogle',)
('Robert Z. Leonard',)
('Charles Craig',)
('Augustus Phillips',)
('Mary Fuller',)
('Kate Bruce',)
('Harry Carey',)
('Henry B. Walthall',)
('Frank Powell',)
('Charles West',)
('Sidney Olcott',)
('Gene Gauntier',)
("Thomas O'Connor",)
('Anna Rosemond',)
(

('Madelyn Clare',)
('Claire Du Brey',)
('Herschel Mayall',)
('Mary Carr',)
('Leslie Austin',)
('Pedro de Cordoba',)
('Zena Keefe',)
('Thomas Holding',)
('Viola Allen',)
('Ben Lyon',)
('Otto Lederer',)
('H.B. Warner',)
('Henry Kolker',)
('Casson Ferguson',)
('Hardee Kirkland',)
('Carol Holloway',)
('Gertrude Claire',)
('Antrim Short',)
('Mary Thurman',)
('William De Vaull',)
('Margaret Campbell',)
('Edwin Stevens',)
('Edna Murphy',)
('Matthew Betz',)
('Montagu Love',)
('Aurora Mardiganian',)
('Gladys George',)
('Jack Conway',)
('Dolores Cassinelli',)
('George MacQuarrie',)
('Hedda Hopper',)
('Eugenie Besserer',)
('William Faversham',)
('Irene Rich',)
('Rhea Mitchell',)
('Jane Lee',)
('Katherine Lee',)
('Zeffie Tilbury',)
('William V. Mong',)
('George Hackathorne',)
('Billy Mason',)
('Marguerite Courtot',)
('Constance Binney',)
('Charles Kent',)
('Winifred Westover',)
('Marguerite Clarlk',)
('Jean Paige',)
('Olive Tell',)
('Bobby Harron',)
('Raymond Cannon',)
('Warner Oland',)
('Lina Cav

('Thelma Hill',)
('James Murray',)
('Monte Montague',)
('Martha Sleeper',)
('Natalie Joyce',)
('Sally Blane',)
('Nina Quartero',)
('Don Alvarado',)
('Sally Eilers',)
('Herbert Stern',)
('Marjorie Beebe',)
('Frank Albertson',)
('Barbara Worth',)
('Buzz Barton',)
('Charles King',)
('Joel McCrea',)
('Jack Oakie',)
('Dorothy Janis',)
('Margaret Mann',)
('George J. Lewis',)
('Ruth Taylor',)
('Rosemary Cooper',)
('Robert Armstrong',)
('Lina Basquette',)
('Tom Keene',)
('Johnny Burke',)
('Betty Caldwell',)
('Ethlyne Clair',)
('June Collyer',)
('Larry Kent',)
('Loretta Young',)
('Doris Dawson',)
('Hugh Trevor',)
('Sammy Cohen',)
('Dorothy Gulliver',)
('Dorothy Burgess',)
('Nora Lane',)
('Roland Drew',)
('Mary Mayberry',)
('Frank Coghlan Jr.',)
('Lane Chandler',)
('Marc MacDermott',)
('Cornelius Keefe',)
('Marjorie King',)
('Don Terry',)
('Ruth Elder',)
('Fanny Brice',)
('Nick Stuart',)
('Sally Phipps',)
('Corliss Palmer',)
('Paul Lukas',)
('Alois Reiser',)
('Ernest Wood',)
('Henry Barrows',)
(

('Scott Kolk',)
('Mary Howard',)
('Joan Davis',)
('Van Heflin',)
('Syd Saylor',)
('Ted Adams',)
('Robert Wilcox',)
('Dick Merrill',)
('Jack Lambie',)
('Virginia Grey',)
('James Bush',)
('Herman Bing',)
('Cora Witherspoon',)
('Spencer Charters',)
('Beatrice Roberts',)
('Joe Sawyer',)
('Joan Valerie',)
('John Merton',)
('Dick Curtis',)
('Leona Roberts',)
('Don Barclay',)
('Martin E. Johnson',)
('Judy Garland',)
('Louise Campbell',)
('Marjorie Weaver',)
('Dorothea Kent',)
('Jane Bryan',)
('Vinton Hayworth',)
('Constance Worth',)
('Karl Hackett',)
('Dame May Whitty',)
('Mary Lawrence',)
('William Stack',)
('Addison Randall',)
('Ralph Cooper',)
('Cleo Herndon',)
('Helen Burgess',)
('Oskar Homolka',)
('Emmett Vogan',)
('Porter Hall',)
('Allyn Joslyn',)
('J. Edward Bromberg',)
('Bill Burrud',)
('Whitney Bourne',)
('Craig Reynolds',)
('John Gallaudet',)
('Katharine Alexander',)
('William Danforth',)
('Vera Ross',)
('Michael Goldstein',)
('Helen Beverley',)
('Louis Stanley',)
('Ray "Crash" Corr

('Susanna Foster',)
('Joan Leslie',)
('Jack LaRue',)
('Mary Healy',)
('Maxie Rosenbloom',)
('Mary Anderson',)
('Mildred Coles',)
('Edward Norris',)
('Arthur Kennedy',)
('Cornel Wilde',)
('Roddy McDowall',)
("Virginia O'Brien",)
('Tommy Dorsey',)
('Bob Crosby',)
('Eddie Bracken',)
('Teresa Wright',)
('Jim Jordan',)
('Sydney Greenstreet',)
('Ben Taggart',)
('Jeanne Kelly',)
('Anne Gwynne',)
('John Sutton',)
('Betty Jane Rhodes',)
('Weaver Brothers',)
('Anna Lee',)
("Martha O'Driscoll",)
('Woody Woodpecker',)
("Edmond O'Brien",)
('Elisabeth Bergner',)
('Edgar Buchanan',)
('Billy Conn',)
('Phillip Terry',)
('Ford Beebe',)
('Hugh Sinclair',)
('Leueen MacGrath',)
('Little Tough Guys',)
('Ruth Ford',)
('Nan Wynn',)
('Fern Emmett',)
('Eddy Waller',)
('Alexis Smith',)
('Jennifer Holt',)
('Veronica Lake',)
('Vivian Leigh',)
('Carol Bruce',)
('Janet Blair',)
('Donald Duck',)
('Peg Leg Pete',)
('Dana Andrews',)
('Cecil Turtle',)
('Wanda McKay',)
('Jinx Falkenburg',)
('Dante the Magician',)
('Charl

...

('ballroom dance',)
('in New York school system',)
('Miranda July',)
('Miles Thompson',)
('Zhang Ziyi',)
('Michelle Yeoh',)
('Stephanie Leonidas',)
('Jason Barry',)
('Rob Brydon',)
('David Monahan',)
('Rupert Friend',)
('Anna Massey',)
('Mathieu Kassovitz',)
('wheelchair rugby',)
('Sean Anders',)
('Allen Zwolle',)
('Shelly Frasier',)
("Q'Orianka Kilcher",)
('Arthur Kane',)
('Hilarie Burton',)
('Faith Ford',)
('Aaron Carter',)
('Bryan Greenberg',)
('Cillian Murphy',)
('Adam Pascal',)
('David Dorfman',)
('clowning and krumping',)
('Wesley Jonathan',)
('Erik Knudsen',)
('Nathan Fillion',)
('Morena Baccarin',)
('Amber Tamblyn',)
('Blake Lively',)
('Michael Angarano',)
('Danielle Panabaker',)
('Mary Elizabeth Winstead',)
('Traylor Howard',)
('Owen Kline',)
('Ian McDiarmid',)
('the 2002',)
('Cory Booker',)
('campaign against',)
('Sharpe James',)
('for mayor of',)
('Newark',)
('New Jersey',)
('Steve Howey',)
('Cameron Richardson',)
('Sophia Bush',)
('Julio Cedillo',)
('Andrea Miller',)
('Mich

('Seth Correa',)
('Marc Saleme',)
('Steve Williams',)
('Quinton Aaron',)
('Brenda Song',)
('Bob Marley',)
('Brian Mahoney',)
('David Ferry',)
('David Della Rocco',)
('Holmes Osborne',)
('Gillian Jacobs',)
('Rinko Kikuchi',)
('Tino Struckmann',)
('Soldiers in the',)
('Greg Robbins',)
('Christina DeMarco',)
('Laura Romeo',)
('Peter Kent',)
('financial crisis of 2007–2010',)
('Emily VanCamp',)
('Christopher Meloni',)
('Chris Massoglia',)
('Michael Cerveris',)
('Al Roker',)
('Josh Stewart',)
('Michael Reilly Burke',)
('Juan Fernández',)
('Kali Hawk',)
('The annual killing of',)
('dolphins',)
('Taiji',)
('Wakayama',)
('Efren Ramirez',)
('Reno Wilson',)
('Arthur Blessitt',)
('Summer Bishil',)
('Shoshana Bush',)
('Damon Wayans, Jr.',)
('Sharlto Copley',)
('Jason Cope',)
('David James',)
('Lorna Raver',)
('Dileep Rao',)
('Adriana Barraza',)
('Joon Park',)
('Eriko Tamura',)
('Jamie Chung',)
('Randall Duk Kim',)
('Tamara Feldman',)
('The casting of the 2006',)
('Broadway',)
('revival',)
('A Chor

('Jeff Foxworthy',)
('Tim Gunn',)
('Wolfgang Puck',)
('Gianna Jun',)
('Li Bingbing',)
('Jiang Wu',)
('Coco Chiang',)
('Jingyu Hu',)
('Archie Kao',)
('Jennifer Lim (voice)',)
('Christina Y. Jun (voice)',)
('Colin Egglesfield',)
('Lorraine Nicholson',)
('Carrie Underwood',)
('Rowan Blanchard',)
('Mason Cook',)
('Alexander Skarsgård',)
('Drew Powell',)
('Andre Royo',)
('Sean Gunn',)
('Stephen Blackehart',)
('Joel Courtney',)
('Gabriel Basso',)
('Riley Griffiths',)
('Ryan Lee',)
('Amanda Michalka',)
('Michael Ian Black',)
('Lauren Conrad',)
('Heidi Montag',)
('Jason Clarke',)
('Chase Ellison',)
('Randy Orton',)
('Alexander Walters',)
('Mia Rose Frampton',)
('Kim Bubbs',)
('Jørgen Langhelle',)
('Jonathan Lloyd Walker',)
('Trond Espen Seim',)
('Stig Henrik Hoff',)
('Carsten Bjørnlund',)
('Jan Gunnar Røise',)
('Paul Braunstein',)
('Tom Hiddleston',)
('Matthew Macfadyen',)
('Gabriella Wilde',)
('Freddie Fox',)
('Rosie Huntington-Whiteley',)
('Tom Sturridge',)
('Frank Grillo',)
('Kurt Angle',)


('Essence Atkins',)
('Dave Sheridan',)
('Emily Alyn Lind',)
('Halley Feiffer',)
('Keegan-Michael Key',)
('Robert Ben Garant',)
('Kumail Nanjiani',)
('Dave Holmes',)
('The actions of the',)
('Catonsville Nine',)
('in the Vietnam War.',)
('Scott Elrod',)
('Dorian Brown',)
('Charles Henry Wyson',)
('Izabela Vidovic',)
("Jim O'Rear",)
('Daniel Emery Taylor',)
('John Dugan',)
('Jason Crowe',)
('April Monique Burril',)
('Genesis Rodriguez',)
('Inbar Lavi',)
('Steven Grayhm',)
('Eddie Hassell',)
('Holland Roden',)
('John Lee Ames',)
('How the concept of virginity shapes the lives of men and women.',)
('Gavin McInnes',)
('Focuses on the drag queen',)
('Jeff "Ja Rule" Atkins',)
('Adrienne Bailon',)
('Vincent Pastore',)
('TobyMac',)
('T-Bone',)
('The tales about voice acting.',)
('Jemma Dallender',)
('Yavor Baharoff',)
('Joe Absolom',)
('Aleksandar Aleksiev',)
('Deragh Campbell',)
('Hannah Gross',)
('Ned Oldham',)
('Kim Taylor',)
('Erin Cummings',)
('Jay Giannone',)
('Weronika Rosati',)
('Eric S

('Mitchell Slaggert',)
('Shannon Purser',)
('Sydney Park',)
('Kevin Hanchard',)
('Fionn Whitehead',)
('Harry Styles',)
('Jack Lowden',)
('Kevin Guthrie',)
('Cara Delevingne',)
('Herbie Hancock',)
('Tiffany Haddish',)
('Mike Colter',)
('Kofi Siriboe',)
('Gethin Anthony',)
('Ty Shelton',)
('Al Gore',)
('Algee Smith',)
('Hannah Murray',)
('Jack Reynor',)
("Ben O'Toole",)
('Kyle Mooney',)
('Menashe Lustig',)
('Ruben Nidorski',)
('Yoel Weisshaus',)
('Meyer Schwartz',)
('Tom Taylor',)
('Abbey Lee Kershaw',)
('Claudia Kim',)
('Sage Correa',)
('Lew Temple',)
('Chris McGinn',)
('Blessin Giraldo',)
('Cori Grainger',)
('Tayla Solomon',)
('Gari McIntyre',)
('Paula Dofat',)
('Kelsey Chow',)
('Seth Rollins',)
('Mo Gallini',)
('Stephanie Sigman',)
('Talitha Bateman',)
('Lulu Wilson',)
('Tayler Buck',)
('Alicia Vela-Bailey',)
('Sebastian Maniscalco',)
('Kari Wahlgren',)
('Ben Safdie',)
('Buddy Duress',)
('Taliah Webster',)
("O'Shea Jackson Jr.",)
('Callum Turner',)
('Élodie Yung',)
('Danielle Macdonal